In [ ]:
!pip install langchain_google_genai

In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [8]:
import os
os.environ["GOOGLE_API_KEY"] = ""

In [12]:
import os
from typing import List, Dict
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import pipeline
from textblob import TextBlob
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class SentimentAnalyzer:
    def __init__(self):
        self.sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

    def analyze(self, text: str) -> Dict:
        sentiment_result = self.sentiment_analyzer(text)[0]
        blob = TextBlob(text)

        return {
            "sentiment": sentiment_result["label"],
            "confidence": sentiment_result["score"],
            "subjectivity": blob.sentiment.subjectivity,
            "polarity": blob.sentiment.polarity
        }

class ViralMetricsAnalyzer:
    def __init__(self):
        # Enhanced viral indicators for Twitter-style content
        self.viral_indicators = {
            "emotional_triggers": ["wild", "insane", "literally crying", "screaming", "based", "real", "unhinged", "no way"],
            "engagement_words": ["ratio", "hot take", "thread", "i'm just saying", "debate me", "fight me"],
            "power_words": ["actually", "literally", "objectively", "factually", "historically", "technically"],
            "meme_phrases": ["ngl", "fr fr", "iykyk", "lowkey", "highkey", "based", "chad", "W", "L"],
            "sass_words": ["bestie", "literally", "imagine", "apparently", "supposedly"]
        }

    def analyze_virality(self, text: str) -> Dict:
        score = 0
        metrics = {
            "emotional_trigger_count": 0,
            "engagement_prompt_count": 0,
            "power_word_count": 0,
            "meme_phrase_count": 0,
            "sass_count": 0
        }

        # Enhanced scoring system
        for trigger in self.viral_indicators["emotional_triggers"]:
            if trigger.lower() in text.lower():
                metrics["emotional_trigger_count"] += 1
                score += 2

        for word in self.viral_indicators["engagement_words"]:
            if word.lower() in text.lower():
                metrics["engagement_prompt_count"] += 1
                score += 1.5

        for word in self.viral_indicators["power_words"]:
            if word.lower() in text.lower():
                metrics["power_word_count"] += 1
                score += 1

        for phrase in self.viral_indicators["meme_phrases"]:
            if phrase.lower() in text.lower():
                metrics["meme_phrase_count"] += 1
                score += 2

        for word in self.viral_indicators["sass_words"]:
            if word.lower() in text.lower():
                metrics["sass_count"] += 1
                score += 1.5

        metrics["virality_score"] = min(score * 10, 100)
        return metrics

class ViralThreadGenerator:
    def __init__(self, google_api_key: str):
        self.google_api_key = google_api_key
        genai.configure(api_key=google_api_key)

        self.llm = GoogleGenerativeAI(model="gemini-pro",
                                    google_api_key=google_api_key)

        self.sentiment_analyzer = SentimentAnalyzer()
        self.viral_analyzer = ViralMetricsAnalyzer()

        self.setup_prompts()
        self.setup_chains()

    def setup_prompts(self):
        # Enhanced prompts for Twitter-style engagement
        self.hook_template = PromptTemplate(
            input_variables=["topic"],
            template="""
            Create a spicy, attention-grabbing first tweet about {topic}.
            Use current Twitter language, sass, and style. Include relevant emojis.
            Make it provocative but substantive. Under 280 characters.
            Add some humor or meme references where appropriate.
            Use phrases like "hot take", "unpopular opinion", "ngl", "fr fr".
            Make it feel like it's written by a chronically online person who's also knowledgeable.
            """
        )

        self.thread_template = PromptTemplate(
            input_variables=["topic", "hook"],
            template="""
            Generate 4 follow-up tweets continuing from this hook about {topic}:
            "{hook}"

            Make the thread:
            1. Use Twitter-style language (bestie, literally, based, etc.)
            2. Include spicy but substantiated takes
            3. Mix humor with actual insights
            4. Present opposing arguments with sass
            5. Use current meme formats and references
            6. End with a mic drop moment
            7. Each tweet under 280 characters
            8. Include relevant emojis and popular Twitter phrases

            Make it feel like a viral tweet that would get both engagement and respect.
            Format as a list of 4 tweets, separated by newlines.
            """
        )

    def setup_chains(self):
        self.hook_chain = LLMChain(
            llm=self.llm,
            prompt=self.hook_template,
            verbose=True
        )

        self.thread_chain = LLMChain(
            llm=self.llm,
            prompt=self.thread_template,
            verbose=True
        )

    def optimize_tweet(self, tweet: str) -> str:
        sentiment_metrics = self.sentiment_analyzer.analyze(tweet)
        viral_metrics = self.viral_analyzer.analyze_virality(tweet)

        if viral_metrics["virality_score"] < 70:
            enhance_prompt = PromptTemplate(
                input_variables=["tweet"],
                template="""
                Make this tweet more engaging using current Twitter style and sass.
                Add meme phrases, engagement hooks, and spicy takes while keeping the core message.
                Keep it under 280 chars and make it feel authentic:
                {tweet}
                """
            )

            enhance_chain = LLMChain(llm=self.llm, prompt=enhance_prompt)
            enhanced_tweet = enhance_chain.run(tweet=tweet)

            new_viral_metrics = self.viral_analyzer.analyze_virality(enhanced_tweet)
            if new_viral_metrics["virality_score"] > viral_metrics["virality_score"]:
                return enhanced_tweet

        return tweet

    def generate_thread(self, topic: str) -> List[Dict]:
        logger.info(f"Generating viral thread about: {topic}")

        try:
            hook = self.hook_chain.run(topic=topic)
            hook = self.optimize_tweet(hook.strip())

            thread_content = self.thread_chain.run(topic=topic, hook=hook)
            tweets = [hook] + thread_content.strip().split('\n')

            optimized_thread = []
            for i, tweet in enumerate(tweets):
                if tweet:
                    optimized_tweet = self.optimize_tweet(tweet)

                    sentiment = self.sentiment_analyzer.analyze(optimized_tweet)
                    viral_metrics = self.viral_analyzer.analyze_virality(optimized_tweet)

                    optimized_thread.append({
                        "position": i + 1,
                        "content": optimized_tweet,
                        "metrics": {
                            "sentiment": sentiment,
                            "viral_potential": viral_metrics
                        }
                    })

            return optimized_thread

        except Exception as e:
            logger.error(f"Error generating thread: {str(e)}")
            raise

def main():
    google_api_key = os.getenv("GOOGLE_API_KEY")
    if not google_api_key:
        raise ValueError("Please set GOOGLE_API_KEY environment variable")

    generator = ViralThreadGenerator(google_api_key)

    print("🔥 Welcome to the Viral Thread Generator 🔥")
    topic = input("Drop your topic bestie (e.g., 'Will Trump win', 'AI taking over'): ")

    print(f"\n🧵 Generating absolute fire thread about: {topic} 🧵\n")

    thread = generator.generate_thread(topic)

    for tweet in thread:
        print(f"\n📱 Tweet {tweet['position']}:")
        print("-" * 50)
        print(tweet['content'])
        print("\n📊 Metrics:")
        print(f"Vibe Check: {tweet['metrics']['sentiment']['sentiment']}")
        print(f"Viral Potential: {tweet['metrics']['viral_potential']['virality_score']:.2f}/100")
        print(f"Sass Level: {tweet['metrics']['viral_potential'].get('sass_count', 0)}")
        print(f"Meme Count: {tweet['metrics']['viral_potential'].get('meme_phrase_count', 0)}")
        print("-" * 50)

if __name__ == "__main__":
    main()

🔥 Welcome to the Viral Thread Generator 🔥
Drop your topic bestie (e.g., 'Will Trump win', 'AI taking over'): Elon musk is illuminati

🧵 Generating absolute fire thread about: Elon musk is illuminati 🧵



> Entering new LLMChain chain...
Prompt after formatting:

            Create a spicy, attention-grabbing first tweet about Elon musk is illuminati.
            Use current Twitter language, sass, and style. Include relevant emojis.
            Make it provocative but substantive. Under 280 characters.
            Add some humor or meme references where appropriate.
            Use phrases like "hot take", "unpopular opinion", "ngl", "fr fr".
            Make it feel like it's written by a chronically online person who's also knowledgeable.
            

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

            Generate 4 follow-up tweets continuing from this hook about Elon musk is illuminati:
            "Hot take fr fr: Elon Musk ain't just a billion